# Домашнее задание к занятию 2.1: Деревья решений. Классификация

## Обзор прошедшего занятия

Что мы делали в классе:

**Задание 1**
* строили деревья
* критерии информативности которых написали даже сами
* визуализировали границы принятия решений в 2d
* и рисовали сами деревья


**Задание 2**
* приняли участие в соревновании на Kaggle, переварив кучу текстовых фичей в численные, проведя кросс-валидацию и сделав сабмит


**Задание 3**
* построили руками несколько метрик качества бинарной классификации


**Задание 4**
* использовали их для оценки классификации разделения статей Ведомостей по топикам

*дополнительно было много приятных ништяков. Например, облако слов, мультипоточность в целях парсинга, удобный инструмент для нахождения правильной css разметки, сохранение моделей в статичные файлы, разделение строк на слова и лемматизация этих слов*

## Домашнее задание

**Lvl 1:**

* взять подготовленные раннее данные из задачи **Titanic**, обучиться на них с помощью дерева решений и кросс-валидации и сделать сабмит
* кросс-валидацию желательно сделать сразу по нескольким фичам ( параметр *grid* в *GridSearchCV* )
* определить самые важные фичи
* вывести дерево решений (можете попробовать установить pydot и webgraphviz для отрисовки деревьев внутри ноутбука)

**Результат:** скрины нового сабмита на Kaggle и построенного дерева

**Lvl 2:** (опционально)

* С помощью функций sklearn.metrics.auc, precision, recall составить функцию для расчёта ROC-AUC, ROC-PRC
* Придумать себе интересную задачу на основе данных из интернета =) Спарсить ещё какой-нибудь сайт (не Ведомости) и решить задачу классификации. Делать свои проекты - круто. Если будут - кидайте мне =) [@NikitaKuznetsov](http://t.me/NikitaKuznesov)

In [1]:
import numpy as np
import pandas as pd

In [2]:
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns

In [40]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sample_submission = pd.read_csv('gender_submission.csv')

In [41]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [42]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [43]:
sample_submission.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [44]:
Xtrain = train.drop(['PassengerId', 'Survived'], axis=1)
Xtest = test.drop(['PassengerId'], axis=1)
Xtrain['is_test'] = False
Xtest['is_test'] = True
X = pd.concat([Xtrain, Xtest], axis=0)
X.index = range(len(X))
X.columns = X.columns.str.lower()
X.name = X.name.str.lower()

In [45]:
X.head()

,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,is_test
0,3,"braund, mr. owen harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,False
1,1,"cumings, mrs. john bradley (florence briggs th...",female,38.0,1,0,PC 17599,71.2833,C85,C,False
2,3,"heikkinen, miss. laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,False
3,1,"futrelle, mrs. jacques heath (lily may peel)",female,35.0,1,0,113803,53.1000,C123,S,False
4,3,"allen, mr. william henry",male,35.0,0,0,373450,8.0500,NaN,S,False


In [46]:
X.shape

(1309, 11)

In [47]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 11 columns):
pclass      1309 non-null int64
name        1309 non-null object
sex         1309 non-null object
age         1046 non-null float64
sibsp       1309 non-null int64
parch       1309 non-null int64
ticket      1309 non-null object
fare        1308 non-null float64
cabin       295 non-null object
embarked    1307 non-null object
is_test     1309 non-null bool
dtypes: bool(1), float64(2), int64(3), object(5)
memory usage: 103.6+ KB


In [48]:
y = train.Survived

In [49]:
X['mr'] = X.name.str.contains("mr.").astype(int)
X['mrs'] = X.name.str.contains("mrs.").astype(int)
X['miss'] = X.name.str.contains("miss.").astype(int)
X['name_len'] = X.name.str.len()
X.age.fillna(X.age.median(), inplace=True)
X.fare.fillna(X.fare.median(), inplace=True)
X.sex = (X.sex =='male').astype(int)

In [50]:
X.cabin.fillna('_', inplace=True)
X['cabin_liter'] = X.cabin.str[0]
X.cabin_liter.value_counts()

_    1014
C      94
B      65
D      46
E      41
A      22
F      21
G       5
T       1
Name: cabin_liter, dtype: int64

У embarked недастает всего двух значений, поэтому просто заполняем наиболее вероятным

In [51]:
X.embarked.fillna('S', inplace=True)
X.embarked.value_counts()

S    916
C    270
Q    123
Name: embarked, dtype: int64

In [56]:
X['ticket_type'] =  X.ticket.apply(lambda t: '' if ord(t.split()[0][0]) <= ord('9') else t.split()[0])
X.ticket_type.value_counts()

              957
PC             92
C.A.           46
SOTON/O.Q.     16
W./C.          14
STON/O         14
CA.            12
A/5            12
SC/PARIS       11
CA             10
A/5.           10
F.C.C.          9
SOTON/OQ        8
C               8
S.O.C.          7
S.O./P.P.       7
STON/O2.        7
A/4             6
SC/Paris        5
SC/AH           5
LINE            4
PP              4
F.C.            3
SOTON/O2        3
A.5.            3
A/4.            3
S.C./PARIS      3
A./5.           3
WE/P            2
P/PP            2
W.E.P.          2
SC              2
W/C             1
S.W./PP         1
C.A./SOTON      1
A.              1
Fa              1
LP              1
AQ/3.           1
STON/OQ.        1
SC/A.3          1
S.O.P.          1
S.P.            1
SW/PP           1
A/S             1
AQ/4            1
SC/A4           1
SCO/W           1
SO/C            1
S.C./A.4.       1
A4.             1
Name: ticket_type, dtype: int64

In [57]:
X['ticket_type_len'] = X.ticket_type.str.len()

In [58]:
from sklearn import preprocessing
le_ticket_type = preprocessing.LabelEncoder()
X.ticket_type = le_ticket_type.fit_transform(X.ticket_type)
le_embarked = preprocessing.LabelEncoder()
X.embarked = le_embarked.fit_transform(X.embarked)
le_cabin_liter = preprocessing.LabelEncoder()
X.cabin_liter = le_cabin_liter.fit_transform(X.cabin_liter)

In [59]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 18 columns):
pclass             1309 non-null int64
name               1309 non-null object
sex                1309 non-null int32
age                1309 non-null float64
sibsp              1309 non-null int64
parch              1309 non-null int64
ticket             1309 non-null object
fare               1309 non-null float64
cabin              1309 non-null object
embarked           1309 non-null int64
is_test            1309 non-null bool
mr                 1309 non-null int32
mrs                1309 non-null int32
miss               1309 non-null int32
name_len           1309 non-null int64
cabin_liter        1309 non-null int64
ticket_type        1309 non-null int64
ticket_type_len    1309 non-null int64
dtypes: bool(1), float64(2), int32(4), int64(8), object(3)
memory usage: 154.8+ KB
